In [13]:
from vivit import ViViT
import torch
import numpy as np
from torch  import nn
from torch.utils.data import Dataset
import pickle
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import random
import torchvision
import os

In [14]:
class skeleton(nn.Module):
    def __init__(self):
        super(skeleton,self).__init__()
        self.project = nn.Linear(34,100)
    def forward(self,x):
        return self.project(x)

In [15]:
class final_head(nn.Module):
    def __init__(self):
        super(final_head,self).__init__()
        self.final = nn.Linear(100,18)
    def forward(self,x):
        self.final(x)

In [4]:
skeleton_data = torch.randn(10,34)
sk_model = skeleton()

In [26]:
print(sk_model)

skeleton(
  (project): Linear(in_features=34, out_features=100, bias=True)
)


In [5]:
#image_size, patch_size, num_classes, num_frames

img = torch.ones([10,16, 3, 224, 320])
model = ViViT(320, 8, 100, 16)
# parameters = filter(lambda p: p.requires_grad, model.parameters())
# parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
# print('Trainable Parameters: %.3fM' % parameters)

out = model(img)

print("Shape of out :", out.shape)      # [B, num_classes]

Shape of out : torch.Size([10, 100])


assets/  module.py     README.md    Test.ipynb
LICENSE  __pycache__/  temporal.py  vivit.py


In [16]:
short_data = "../test/all_data-hrc.pickle"
data = pickle.load(open(short_data,'rb'))

In [104]:
len(data["class_ix"].keys())

13

In [ ]:
torch.tensor([for i in data["all_data"][0]["video_frames"].keys()])

In [ ]:
data["all_data"][0]["video_frames"]

In [49]:
getabuse = [i  for i in data["all_data"] if i["video_file_name"][:5]=="Abuse"]

In [50]:
getabuse[0].keys()

dict_keys(['category_name', 'video_file_name', 'class_idx', 'video_frames', 'person_frame'])

In [139]:
b = torchvision.io.read_video("../UCF_Videos/Abuse/Abuse002_x264.mp4")

In [153]:
class hrc(Dataset):
    #just for abuse
    def __init__(self, pickl_list,n_frames):
        self.all_data = pickl_list["all_data"]
        self.getabuse = [i  for i in self.all_data if i["video_file_name"][:5]=="Abuse"]
        self.n_frames = n_frames #n_frames to extract from the data and video
        self.ix_class = pickl_list["ix_class"]
        self.class_ix = pickl_list["class_ix"]
        
    def __len__(self):
        return len(self.getabuse)
    
    
    def __getitem__(self,idx):
        video_class = self.getabuse[idx]["video_file_name"]
        read_video = torchvision.io.read_video(f"../UCF_Videos/Abuse/{video_class}_x264.mp4")
        # print(read_video[0].size())
        trajectory = self.getabuse[idx]["video_frames"] #get trajectory of video    
        get_trajectory_frames = np.array(list(trajectory.keys()))
        min_frame = min(get_trajectory_frames) 
        max_frame = max(get_trajectory_frames)

        frame_start = random.randint(min_frame,max_frame-self.n_frames)
        closest_trajectory_ix = np.argmin(abs(frame_start-get_trajectory_frames))
        segment_trajectory = get_trajectory_frames[closest_trajectory_ix:closest_trajectory_ix+self.n_frames]

        segment_trajectory = torch.Tensor([j for i in segment_trajectory for j in trajectory[i].values()])
        final_video = read_video[0][frame_start:frame_start+self.n_frames]/255
        final_video = rearrange(final_video,"f h w c -> f c h w")
        return (final_video)

In [99]:
# [j for i in range(5) for j in range(5)]

In [154]:
a = hrc(data,30)

In [93]:
# segment = a[1][1]

In [147]:
# frames,trajectory = a[0]


In [148]:
a[2][0].shape

torch.Size([30, 3, 240, 320])

In [107]:
frames = frames.unsqueeze(0)

In [124]:
frames = rearrange(frames,"b f h w c -> b f c h w")/255

In [155]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(a, batch_size=5, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [156]:
for i in train_dataloader:
    print(i.shape)
    break

torch.Size([5, 30, 3, 240, 320])


In [ ]:
next(iter(train_dataloader))

In [126]:
#image_size, patch_size, num_classes, num_frames
# img = torch.ones([10,30, 3, 224, 320])

model = ViViT(320, 16, 13, 30)
# parameters = filter(lambda p: p.requires_grad, model.parameters())
# parameters = sum([np.prod(p.size()) for p in parameters]) / 1_000_000
# print('Trainable Parameters: %.3fM' % parameters)

out = model(frames)

In [127]:
print(out)

tensor([[-0.5124, -0.0861,  0.1992,  0.0496,  1.1429,  0.3310,  0.2900, -0.3958,
         -1.0374,  0.3132, -0.2199, -0.0834,  0.4373]],
       grad_fn=<AddmmBackward0>)
